<a href="https://colab.research.google.com/github/davidwhogg/TargetSelection/blob/master/ipynb/catalog_likelihood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib
import pylab as plt
!pip install emcee
import emcee
!pip install corner
import corner
%matplotlib inline
np.random.seed(42)

In [0]:
# make a log-normal universe of uncensored sources
def make_universe(N, mean=0.5, sigma=1.):
  x = np.exp(sigma * (mean + np.random.normal(size=N)))
  return x

def ln_censoring(xs, cutoff):
  ys = np.zeros_like(xs)
  ys[xs < cutoff] = -np.inf
  return ys

def censoring(xs, cutoff):
  return np.exp(ln_censoring(xs, cutoff))

# censoring to make catalog
def make_catalog(xs, cutoff=3.):
  return xs[np.log(np.random.uniform(size=len(xs))) < ln_censoring(xs, cutoff)]

In [0]:
  # make the universe and a histogram of it
u_xs = make_universe(8192)
binwidth = 0.1
bins = np.arange(0., 8., binwidth)
uy, foo = plt.histogram(u_xs, bins=bins)
xbins = 0.5 * (bins[1:] + bins[:-1])

In [0]:
# plot the universe
plt.clf()
plt.plot(xbins, uy, "k.")
plt.semilogy()
plt.xlim(0., 8.)
plt.title("universe")

In [0]:
# censor the universe to make the catalog
c_xs = make_catalog(u_xs)
cy, foo = plt.histogram(c_xs, bins=bins)
print(c_xs.shape, np.max(c_xs))

In [0]:
# plot the catalog
plt.clf()
plt.plot(xbins, cy, "k.")
plt.semilogy()
plt.xlim(0, 8.)
plt.title("catalog")

In [0]:
# functions for use in the likelihood
def ln_lognormal(xs, mean, sigma):
  lnxs = np.log(xs)
  return -0.5 * (lnxs - mean) ** 2 / sigma ** 2 - 0.5 * np.log(2. * np.pi * sigma ** 2) - lnxs

def lognormal(xs, mean, sigma):
  return np.exp(ln_lognormal(xs, mean, sigma))

def censored_model(pars, xs):
  lnamp, mean, sigma, cutoff = pars
  return np.exp(lnamp + ln_lognormal(xs, mean, sigma) + ln_censoring(xs, cutoff))

# integrate the censored model
# HACK MAGIC: fixed integration grid; that's bad
def expected_number(pars):
  dx = 0.01
  xs = np.arange(0. + 0.5 * dx, 64., dx)
  ys = censored_model(pars, xs)
  return dx * np.sum(ys)

# likelihood function, with switch to make it Loredo-like or not
def ln_like(pars, xs, Loredo):
  lnamp, mean, sigma, cutoff = pars
  amp = np.exp(lnamp)
  if Loredo:
    sum_term = np.sum(lnamp + ln_lognormal(xs, mean, sigma))
  else:
    sum_term = np.sum(lnamp + ln_lognormal(xs, mean, sigma) + ln_censoring(xs, cutoff))
  return sum_term - expected_number(pars)

# prior function
def ln_prior(pars):
  lnamp, mean, sigma, cutoff = pars
  if lnamp < 5.:
    return -np.inf
  if lnamp > 13.:
    return -np.inf
  if mean < -5.:
    return -np.inf
  if mean > 5.:
    return -np.inf
  if sigma < 0.1:
    return -np.inf
  if sigma > 3.:
    return -np.inf
  if cutoff < 0.001:
    return -np.inf
  if cutoff > 5.:
    return -np.inf
  return 0.

# posterior function
def ln_post(pars, data, Loredo):
  lnp = ln_prior(pars)
  if np.isfinite(lnp):
    lnp += ln_like(pars, data, Loredo)
  return lnp

In [0]:
pars0 = [np.log(8192), -0.5, 1.0, 2.999]
print(ln_post(pars0, c_xs, True))
print(ln_post(pars0, c_xs, False))
print(expected_number(pars0))

In [0]:
# Sample the DFM LF case
nsteps = 512
nwalkers = 64
ndim = len(pars0)
pos = pars0 + 1e-4*np.random.normal(size=(nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_post, args=(c_xs, False))
sampler.run_mcmc(pos, nsteps);
# take only the second half
DFM_flatchain = sampler.chain[:,-(nsteps // 2):,:].reshape((nwalkers * nsteps // 2), ndim)

In [0]:
# Plot the DFM LF case
import corner
figure = corner.corner(DFM_flatchain[-(nsteps * nwalkers // 2):,:])

In [0]:
# plot some DFM samples with the data
nsamples, ndim = DFM_flatchain.shape
dx = 0.001
plotxs = np.arange(0. + 0.5 * dx, 32., dx)
plt.clf()
js = np.argsort(np.random.uniform(size=nsamples))[0:16]
for j in js:
  plotys = censored_model(DFM_flatchain[j], plotxs)
  plt.plot(plotxs, plotys, "r-", alpha=0.2)
plt.step(xbins, cy / binwidth, color="k", where="mid", alpha=0.75)
plt.xlim(0., 8.)
plt.title("DFM case")

In [0]:
# Sample the Loredo LF case
nsteps = 512
nwalkers = 64
ndim = len(pars0)
pos = pars0 + 1e-4*np.random.normal(size=(nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_post, args=(c_xs, True))
sampler.run_mcmc(pos, nsteps);
# take only the second half
Loredo_flatchain = sampler.chain[:,-(nsteps // 2):,:].reshape((nwalkers * nsteps // 2), ndim)

In [0]:
# Corner-plot the Loredo LF case
import corner
figure = corner.corner(Loredo_flatchain[-(nsteps * nwalkers // 2):,:])

In [0]:
# plot some Loredo samples with the data
nsamples, ndim = Loredo_flatchain.shape
dx = 0.001
plotxs = np.arange(0. + 0.5 * dx, 32., dx)
plt.clf()
js = np.argsort(np.random.uniform(size=nsamples))[0:16]
for j in js:
  plotys = censored_model(Loredo_flatchain[j], plotxs)
  plt.plot(plotxs, plotys, "r-", alpha=0.2)
plt.step(xbins, cy / binwidth, color="k", where="mid", alpha=0.75)
plt.xlim(0., 8.)
plt.title("Loredo case")